In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jackaduma/SecBERT")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/378k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("jackaduma/SecBERT")


model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import random

# Load SecBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("jackaduma/SecBERT")
model = AutoModelForMaskedLM.from_pretrained("jackaduma/SecBERT")

def augment_text(sentence, num_augments=3, mask_prob=0.3, top_k=5):
    """
    Augments a sentence using SecBERT by randomly masking words and predicting replacements.

    Parameters:
    - sentence (str): The input sentence to augment.
    - num_augments (int): Number of augmented sentences to generate.
    - mask_prob (float): Probability of masking each word in the sentence.
    - top_k (int): Number of top predictions to consider for replacement.

    Returns:
    - List of augmented sentences.
    """
    tokens = tokenizer.tokenize(sentence)
    num_masks = max(1, int(len(tokens) * mask_prob))  # Ensure at least one word is masked
    masked_sentences = []

    for _ in range(num_augments):
        # Randomly select words to mask
        masked_tokens = tokens.copy()
        mask_indices = random.sample(range(len(tokens)), num_masks)

        for idx in mask_indices:
            masked_tokens[idx] = tokenizer.mask_token  # Replace with [MASK]

        masked_text = tokenizer.convert_tokens_to_string(masked_tokens)
        input_ids = tokenizer(masked_text, return_tensors="pt")["input_ids"]

        # Predict masked words
        with torch.no_grad():
            outputs = model(input_ids)
            predictions = outputs.logits

        # Replace masked words with predicted tokens
        for idx in mask_indices:
            masked_idx = (input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]
            predicted_ids = torch.topk(predictions[0, masked_idx], top_k).indices
            predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)

            # Choose a random prediction from top_k choices
            replacement = random.choice(predicted_tokens)
            masked_tokens[idx] = replacement

        augmented_sentence = tokenizer.convert_tokens_to_string(masked_tokens)
        masked_sentences.append(augmented_sentence)

    return masked_sentences

# Example Usage
input_sentence = "I had continue received random calls and abusive messages in my whatsapp Someone added my number in a unknown facebook group name with  Only Girls  and still getting calls from unknown numbers pls help me and sort out the issue  as soon as possible Thank you."
augmented_sentences = augment_text(input_sentence, num_augments=5, mask_prob=0.3, top_k=5)

for i, aug in enumerate(augmented_sentences):
    print(f"Augmented {i+1}: {aug}")


Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Augmented 1: i had recently received random calls and abusive messages he my whatsapp previously previously my number previously a unknown facebook group name he previously girls since since getting calls from recently previously pls help me previously sort out the issue as soon he possible thank since.
Augmented 2: phone had we received phone calls and abusive messages he my whatsapp we you my we in a he facebook group name with only i you still getting i from unknown numbers pls help me and sort out i issue as soon as possible thank you phone
Augmented 3: i can continue will would calls and abusive messages in my whatsapp someone added my number would a unknown facebook will can with only girls and would getting can from would numbers pls to me and had would the issue to had as possible thank you.
Augmented 4: i had continue received random calls to abusive messages in disguised whatsapp someone disguised, and in a disguised facebook group to with, asking and still getting asking fro

In [ ]:
import re
import nltk
from nltk.corpus import words
from collections import Counter

# Download NLTK words corpus (Run this only once)
nltk.download("words")
english_vocab = set(words.words())

### 1️⃣ Gibberish Word Detection (Regex-based)
def is_gibberish_regex(word):
    """Detect gibberish using regex (long non-dictionary words)."""
    return bool(re.fullmatch(r"[a-zA-Z]{10,}", word))  # Adjust threshold if needed

### 2️⃣ Dictionary-Based Gibberish Detection
def is_gibberish_word(word):
    """Check if a word exists in the English dictionary."""
    return word.lower() not in english_vocab

### 3️⃣ Character Frequency-Based Gibberish Detection
def character_distribution(word):
    """Measure letter frequency compared to normal English."""
    english_freq = "etaoinshrdlcumwfgypbvkjxqz"  # Common English letters
    word = word.lower()
    letter_counts = Counter(word)
    score = sum(letter_counts[char] for char in english_freq if char in letter_counts)
    return score / len(word) if word else 0  # Normalize

def is_gibberish_distribution(word, threshold=0.2):
    """Detect gibberish based on letter frequency distribution."""
    return character_distribution(word) < threshold

### 4️⃣ Final Gibberish Word Removal Function
def remove_gibberish(text):
    """Removes gibberish words while keeping valid English words."""
    words = text.split()  # Tokenize by spaces (no need for punctuation)
    cleaned_words = []

    for word in words:
        if is_gibberish_regex(word) or is_gibberish_word(word) or is_gibberish_distribution(word):
            continue  # Remove gibberish words
        cleaned_words.append(word)

    return " ".join(cleaned_words)  # Reconstruct sentence without gibberish

# 🧪 Test Case (Passing running text directly)
test_text = """FRAUD THROUGH  UPI PHONE PE
 BANK  UCO BANK
 ACCOUNT NUMBER
 TOTAL AMOUNT
 PLEASE HOLD REVERSE TOTAL AMOUNT IN THE COMPLAINT ACCOUNT AND TAKE NECESSARY ACTION"""

# Run test
cleaned_text = remove_gibberish(test_text)
print(cleaned_text)


FRAUD THROUGH PHONE BANK BANK ACCOUNT NUMBER TOTAL AMOUNT PLEASE HOLD REVERSE TOTAL AMOUNT IN THE COMPLAINT ACCOUNT AND TAKE NECESSARY ACTION


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import random

# Load SecBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("jackaduma/SecBERT")
model = AutoModelForMaskedLM.from_pretrained("jackaduma/SecBERT")

def correct_gibberish(sentence, mask_prob=0.3, top_k=5):
    """
    Detects and corrects gibberish words using SecBERT by masking and predicting replacements.

    Parameters:
    - sentence (str): Input sentence to clean.
    - mask_prob (float): Probability of masking potential gibberish words.
    - top_k (int): Number of top predictions to consider for replacement.

    Returns:
    - Cleaned sentence with gibberish words replaced.
    """
    tokens = tokenizer.tokenize(sentence)
    num_masks = max(1, int(len(tokens) * mask_prob))  # Ensure at least one word is masked
    masked_tokens = tokens.copy()

    # Randomly select words to mask
    mask_indices = random.sample(range(len(tokens)), num_masks)

    for idx in mask_indices:
        masked_tokens[idx] = tokenizer.mask_token  # Replace with [MASK]

    masked_text = tokenizer.convert_tokens_to_string(masked_tokens)
    input_ids = tokenizer(masked_text, return_tensors="pt")["input_ids"]

    # Predict masked words
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # Replace masked words with predicted tokens
    for idx in mask_indices:
        masked_idx = (input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1][0]
        predicted_ids = torch.topk(predictions[0, masked_idx], top_k).indices
        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)

        # Choose a random prediction from top_k choices
        replacement = random.choice(predicted_tokens)
        masked_tokens[idx] = replacement

    cleaned_sentence = tokenizer.convert_tokens_to_string(masked_tokens)
    return cleaned_sentence

# 🧪 **Test Case**
input_sentence = """Httpswwwxnxxtvvideousapbfuckkkarrr Above link is correct video URl she is one of our relative Please help her now she is going to get marry other person they are not aware of this information while its recording some malwere functionality has recorded without their permission Please help her and remove the above video"""

# Run gibberish correction
cleaned_sentence = correct_gibberish(input_sentence, mask_prob=0.3, top_k=5)
print("🔹 Cleaned Sentence:", cleaned_sentence)


Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


🔹 Cleaned Sentence: httpswww named namedxtvvideousapbfuckkkarrr method shimrat is correct named shimrat she is one of shimrat relative please help her now she is shimrat to get / / ftp person they are shimrat ftp of shimrat information while its / some malwere named shimrat recorded without method permission please help her named remove named above video
